<a href="https://colab.research.google.com/github/illhyhl1111/SNU_ML2019/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# original code from : https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# LSTM 예시
 * input dimension과 output dimension이 3인 LSTMTagger
 * lstm(input, hidden) 은 해당 input과 주어진 현재 hidden state를 받아,
   output과 다음 hidden state를 리턴한다.
 * lstm(inputs, hidden) 은 해당 input sequence를 주어진 현재 state에 차례로 넣어,
   output sequence와 최종 hidden state를 리턴한다.

In [7]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    print(out)
    print(hidden)

inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.1177, -0.0202,  0.1327]]], grad_fn=<StackBackward>)
(tensor([[[-0.1177, -0.0202,  0.1327]]], grad_fn=<StackBackward>), tensor([[[-0.2304, -0.0818,  0.6783]]], grad_fn=<StackBackward>))
tensor([[[-0.0806,  0.1281,  0.1871]]], grad_fn=<StackBackward>)
(tensor([[[-0.0806,  0.1281,  0.1871]]], grad_fn=<StackBackward>), tensor([[[-0.2258,  0.2400,  0.4334]]], grad_fn=<StackBackward>))
tensor([[[-0.2598,  0.1070,  0.1443]]], grad_fn=<StackBackward>)
(tensor([[[-0.2598,  0.1070,  0.1443]]], grad_fn=<StackBackward>), tensor([[[-0.4415,  0.2775,  0.3539]]], grad_fn=<StackBackward>))
tensor([[[-0.6068,  0.0427,  0.0903]]], grad_fn=<StackBackward>)
(tensor([[[-0.6068,  0.0427,  0.0903]]], grad_fn=<StackBackward>), tensor([[[-0.8996,  0.2223,  0.4447]]], grad_fn=<StackBackward>))
tensor([[[-0.5103,  0.0466,  0.1605]]], grad_fn=<StackBackward>)
(tensor([[[-0.5103,  0.0466,  0.1605]]], grad_fn=<StackBackward>), tensor([[[-0.7977,  0.1429,  0.3695]]], grad_fn=<StackBackward>))
tensor([[[

# 데이터 예시
 * 각 단어의 품사를 추론하는 모델을 만드는 것을 목표로 한다.
 * 각 단어 별로 품사가 labeling된 데이터를 준비한다(아래 코드의 training_data).
 * 각 단어 혹은 품사 sequence를 숫자로 encoding한다(아래 코드의 prepare_sequence 함수).
 * 이 때, 단어를 숫자로 encoding할 때는 word_to_ix dictionary를 사용하고, 품사를 숫자로 encoding할 때는 tag_to_ix dictionary를 사용한다.

In [4]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


# 품사 예측 LSTM 모델
 * 각 단어를 embedding_dim 차원 공간에 embedding하여, 해당 embedding을 LSTM의 input으로 사용한다.
 * LSTM은 hidden_dim 차원의 hidden state를 가지며, output인 품사 score는 hidden state에 fully connected layer를 적용하여 얻을 수 있다.

In [0]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

# LSTM training
 * 아까 정의한 prepare_sequence 함수를 이용해 training data를 encoding하여 LSTM에 input으로 넣는다.
 * LSTM 모델에서 나온 tag score와 tag label와의 NLLLoss를 loss로써 사용하여 학습한다.
  (https://pytorch.org/docs/stable/nn.html#nllloss 참조)

In [6]:
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-1.3599, -0.9680, -1.0121],
        [-1.2462, -0.9999, -1.0656],
        [-1.3484, -0.9154, -1.0789],
        [-1.4136, -0.8850, -1.0670],
        [-1.4271, -0.8878, -1.0543]])
tensor([[-0.0437, -3.4309, -4.5645],
        [-4.1781, -0.0296, -4.2791],
        [-3.4239, -2.8304, -0.0960],
        [-0.0422, -4.5994, -3.4646],
        [-4.8367, -0.0203, -4.4117]])


 * 각 단어에 대해 가장 score가 높은 index에 해당되는 품사가 LSTM이 추론한 품사이다.
 * 학습한 결과는 'The dog ate the apple'이라는 문장에 대해 'DET NN V DET NN'으로 품사를 추론하고, 'Everybody read that book'이라는 문장에 대해 'NN V DET NN'으로 품사를 추론함을 알 수 있다.